In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# New Section

In [0]:
import numpy as np
import glob
import matplotlib.pyplot as plt
from skimage.color import rgb2grey
from matplotlib.image import imread
#!pip install -q keras
from keras.models import Sequential
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation, ReLU, Dropout
from keras.utils import to_categorical

class_names = ['violin', 'viola', 'cello', 'double-bass', 'guitar', 'clarinet', 'saxophone']
number_of_classes = len(class_names)

instruments = {'violin': 1, 'viola': 2, 'cello': 3, 'double-bass': 4,
               'guitar': 5, 'clarinet': 6, 'saxophone': 7}

files = glob.glob('/content/drive/My Drive/Colab Notebooks/spectograms/*.png')
np.random.shuffle(files)

data, labels = [], []

loaded = 0

print('Number of samples {0}'.format(len(files)))

for file in files:
  img, name = imread(file), file.split('/')[-1].split('_')[0]
  data.append(rgb2grey(img))
  labels.append(instruments[name] - 1)

  loaded += 1
  if loaded % 200 == 0:
    print(loaded)

  if loaded == 5000:
    break

data, labels = np.array(data), np.array(labels)

s_60p, s_20p = int(len(data) * 0.6), int(len(data) * 0.2)

X_train, y_train = data[:s_60p], labels[:s_60p] #60%
X_validation, y_validation = data[s_60p: s_60p + s_20p], labels[s_60p: s_60p + s_20p]  #20%
X_test, y_test = data[s_60p + s_20p:], labels[s_60p + s_20p:] #20%

X_train = X_train.reshape(X_train.shape[0], 204, 800, 1)
X_test = X_test.reshape(X_test.shape[0], 204, 800, 1)
X_validation = X_validation.reshape(X_validation.shape[0], 204, 800, 1)

print('Dataset has been loaded succesfully.')
print(X_train.shape, y_train.shape)
print(X_validation.shape, y_validation.shape)

print(to_categorical(y_train).shape)
print(to_categorical(y_validation).shape)
print(X_train.shape[1:])

epochs = 30

model = Sequential([
        Conv2D(32, (3, 3), input_shape=(204, 800, 1)),
        Activation('relu'),
        Conv2D(32, (3, 3)),
        Activation('relu'),
        MaxPooling2D(pool_size=(3, 3)),
        Dropout(0.25),

        Conv2D(64, (3,3), padding='same'),
        Activation('relu'),
        Conv2D(64, (3, 3)),
        Activation('relu'),
        MaxPooling2D(pool_size=(3, 3)),
        Dropout(0.25),

        Conv2D(128, (3, 3), padding='same'),
        Activation('relu'),
        Conv2D(128, (3, 3)),
        Activation('relu'),
        MaxPooling2D(pool_size=(3, 3)),
        Dropout(0.25),

        Flatten(),
        Dense(128),
        Activation('relu'),
        Dropout(0.5),
        Dense(number_of_classes),
        Activation('softmax')
    ])

model.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

print('Fitting started...')
history = model.fit(X_train, to_categorical(y_train), epochs=epochs, validation_data=(X_validation, to_categorical(y_validation)))

#model accuracy
print(history.history)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valid'], loc='upper left')
plt.show()

#model loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valid'], loc='upper left')
plt.show()



Number of samples 5919
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
Dataset has been loaded succesfully.
(3000, 204, 800, 1) (3000,)
(1000, 204, 800, 1) (1000,)
(3000, 7)
(1000, 7)
(204, 800, 1)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 202, 798, 32)      320       
_________________________________________________________________
activation_1 (Activation)    (None, 202, 798, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 200, 796, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 200, 796, 32)      0 

In [3]:

scores = model.evaluate(X_test, to_categorical(y_test), verbose=1)
print(scores)

NameError: ignored